In [3]:
import os
import pandas as pd
import numpy as np
import random

# -------------------------------------------------------------------
#                         CONFIGURATION
# -------------------------------------------------------------------

CONT_VARS = [
    "HR", "O2Sat", "SBP", "DBP", "Resp", "Temp", "Lactate",
    "WBC", "Hct", "Hgb", "Platelets", 
]

OUTPUT_FOLDER = "synthetic_patients"

# -------------------------------------------------------------------
#                     LOAD REAL PATIENTS ONCE
# -------------------------------------------------------------------

def load_real_patients(folder):
    """
    Loads all .psv ICU patient files ONCE.
    Returns: [list_of_dataframes], [list_of_patient_ids]
    """
    patient_dfs = []
    patient_ids = []

    for f in os.listdir(folder):
        if f.endswith(".psv"):
            full = os.path.join(folder, f)
            df = pd.read_csv(full, sep="|")
            patient_dfs.append(df)
            patient_ids.append(f.replace(".psv", ""))

    print(f"Loaded {len(patient_dfs)} real patient files into memory.")
    return patient_dfs, patient_ids

# -------------------------------------------------------------------
#               VECTORIZED TIME WARPING OF SIGNALS
# -------------------------------------------------------------------

def apply_time_warp(df, magnitude=0.08):
    """
    Applies small Gaussian noise to continuous vitals.
    Vectorized across all rows and columns.
    """
    warped = df.copy()
    mask = warped[CONT_VARS].notna()

    noise = np.random.normal(0, magnitude, size=(len(df), len(CONT_VARS)))
    warped.loc[:, CONT_VARS] = warped.loc[:, CONT_VARS] * (1 + noise)

    # Ensure we do not introduce NaNs where values existed
    warped.loc[:, CONT_VARS] = warped.loc[:, CONT_VARS].where(mask)

    return warped

# -------------------------------------------------------------------
#                VECTORIZED PATIENT MERGING (MixUp style)
# -------------------------------------------------------------------

def align_times(df1, df2):
    """Trim to equal length for vectorized merging."""
    m = min(len(df1), len(df2))
    return df1.iloc[:m].reset_index(drop=True), df2.iloc[:m].reset_index(drop=True)

def merge_patients(base_df, donor_df, alpha=0.3):
    """
    Vectorized convex combination of vitals.
    Must have the same length (use align_times).
    """
    base_df, donor_df = align_times(base_df, donor_df)

    merged = base_df.copy()
    w = np.random.uniform(alpha, 1 - alpha)

    merged[CONT_VARS] = (
        w * base_df[CONT_VARS].values +
        (1 - w) * donor_df[CONT_VARS].values
    )

    return merged

# -------------------------------------------------------------------
#               INSERT REALISTIC SEPSIS PHYSIOLOGY RAMP
# -------------------------------------------------------------------

def insert_sepsis_ramp(df, final_prob=1.0, ramp_hours=6):
    """
    Add an early-warning sepsis trajectory:
        - HR ↑
        - RR ↑
        - Temp ↑
        - SBP ↓ (shock)
        - O2Sat ↓
        - WBC ↑ or ↓ (both happen clinically)
    Creates a ramp over the last N hours.
    """
    df = df.copy()
    n = len(df)

    onset = random.randint(max(2, n - ramp_hours), n - 1)
    ramp = np.linspace(0, 1, n - onset)

    # vectorized physiologic changes
    df.loc[onset:, "HR"] *= (1 + 0.25 * ramp)
    df.loc[onset:, "Resp"] *= (1 + 0.20 * ramp)
    df.loc[onset:, "Temp"] *= (1 + 0.02 * ramp)
    df.loc[onset:, "SBP"] *= (1 - 0.15 * ramp)
    df.loc[onset:, "O2Sat"] *= (1 - 0.05 * ramp)

    # WBC can go up or down (in line with sepsis)
    if random.random() < 0.5:
        df.loc[onset:, "WBC"] *= (1 + 0.40 * ramp)
    else:
        df.loc[onset:, "WBC"] *= (1 - 0.40 * ramp)

    # Assign labels
    df["SepsisLabel"] = 0
    df.loc[onset:, "SepsisLabel"] = 1

    return df

# -------------------------------------------------------------------
#                        FABRICATE ONE PATIENT
# -------------------------------------------------------------------

def fabricate_sepsis_patient(real_dfs):
    """
    Creates one synthetic patient by:
        1. randomly picking a base patient
        2. warping vitals
        3. merging with a donor patient
        4. inserting a sepsis onset trajectory
    """

    base_df = random.choice(real_dfs)
    donor_df = random.choice(real_dfs)

    # Step 1 — warp
    warped = apply_time_warp(base_df)

    # Step 2 — merge (MixUp-style)
    mixed = merge_patients(warped, donor_df)

    # Step 3 — sepsis ramp
    synthetic = insert_sepsis_ramp(mixed)

    return synthetic

def fabricate_healthy_patient(real_dfs):
    """
    Generates a synthetic non-septic patient.
    Uses warping + merging but no sepsis trajectory.
    """
    base_df = random.choice(real_dfs)
    donor_df = random.choice(real_dfs)

    # 1) small physiologic variability
    warped = apply_time_warp(base_df, magnitude=0.04)

    # 2) light merging
    mixed = merge_patients(warped, donor_df, alpha=0.2)

    # 3) ensure no sepsis labels
    mixed["SepsisLabel"] = 0

    return mixed


# -------------------------------------------------------------------
#                       WRITE PSV TO DISK
# -------------------------------------------------------------------

def save_patient(df, out_folder, patient_id):
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    out_path = os.path.join(out_folder, f"{patient_id}.psv")
    df.to_csv(out_path, sep="|", index=False)

# -------------------------------------------------------------------
#                      MAIN GENERATION FUNCTION
# -------------------------------------------------------------------

def generate_dataset(
    source_folder,
    out_folder="synthetic_patients",
    count=200
):
    real_dfs, _ = load_real_patients(source_folder)

    half = count // 2

    print(f"Generating {half} septic and {half} healthy synthetic patients...")

    for i in range(half):
        syn_df = fabricate_sepsis_patient(real_dfs)   # septic
        save_patient(syn_df, out_folder, f"synthetic_septic_{i:04d}")

    for i in range(half):
        syn_df = fabricate_healthy_patient(real_dfs)  # healthy
        save_patient(syn_df, out_folder, f"synthetic_healthy_{i:04d}")

    print("Done")


# -------------------------------------------------------------------
#                             EXECUTE
# -------------------------------------------------------------------

if __name__ == "__main__":
    generate_dataset(
        source_folder="PhysioData/training/training_setA",
        out_folder="synthetic_patients",
        count=2000
    )


Loaded 20336 real patient files into memory.
Generating 1000 septic and 1000 healthy synthetic patients...


C:\Users\Mitchell Esbenshade\AppData\Local\Temp\ipykernel_8628\2394059506.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 78.65242119  70.86257774  78.37288086  81.17038508  78.35289751
  93.97058489  93.25362934  76.14125608  67.92247946  89.57561109
  79.65521975  76.69361624  77.397407    82.53150131  80.70383562
  72.25734623  82.3151872   92.77050639  73.84507702  85.42265281
  93.2067548   91.65735213  85.50068386  82.28187365  79.50102086
 100.91196999]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warped.loc[:, CONT_VARS] = warped.loc[:, CONT_VARS] * (1 + noise)
C:\Users\Mitchell Esbenshade\AppData\Local\Temp\ipykernel_8628\2394059506.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[106.8111448   92.74002916 101.11475188 103.21677589 103.51507772
  92.53215455  93.30341625  96.0698

Done
